return an df
sent_file_name, note_file_name, note_file_line_index_start, note_file_line_index_end
sent_1.bio,note_100.bio,5,12
sent_2.bio,note_100.bio,6,18

In [27]:
import os
import pandas as pd
import joblib

In [28]:
note_folder = '/home/gy237/project/EHR_Foundation_model/note_2_sentence_level/note_level'
sent_folder = '/home/gy237/project/EHR_Foundation_model/note_2_sentence_level/sent_level'

note_files = os.listdir(note_folder)
note_files = [i for i in note_files if i.endswith('.bio')]
print(len(note_files))

sent_files = os.listdir(sent_folder)
sent_files = [i for i in sent_files if i.endswith('.bio')]
print(len(sent_files))

163
1004


In [29]:
notes = []
for file in note_files:
    with open(f'{note_folder}/{file}', 'r') as f:
        lines = f.readlines()
    word_list = [i.strip().split('\t')[0] for i in lines]
    word_list = [f'@{i}$' for i in word_list]
    notes.append(''.join(word_list))

# ["HISTORY OF PRESENT ILLNESS : This is a very nice 53 - year - old white male with no previous history of heart disease , was admitted to rule out MI and coronary artery disease .  The patient has history of hypercholesterolemia , presently on Lipitor 20 mg a day and hyperthyroidism , on Synthroid 0 . 088 mg per day .  Also , history of chronic diverticulitis with recent bouts .  The patient has been doing well , seen in my office at the end of December for complete physical examination .  I had ordered a stress test for him , then delayed due to a family illness .  However , denies any chest pain or chest tightness with exertion .  The patient was doing well .  He was watching television yesterday afternoon or p . m . and fell asleep holding his head in his left hand .  He awoke and noticed pain in the jaw and neck area , on both sides , but no shortness of breath , diaphoresis , nausea , or chest pain .  He is able to go to sleep , woke up this morning with same discomfort , decided to call our office , talked to our triage nurse , who instructed to come to the emergency room for possibility of just having a cardiac event .  The patient ' s pain resolved .  He was given nitroglycerin in the emergency room drawing his blood pressure 67 / 32 .  Blood pressure quickly came back to normal with the patient ' s reverse Trendelenburg . "]

In [30]:
sents = []
for file in sent_files:
    with open(f'{sent_folder}/{file}', 'r') as f:
        lines = f.readlines()
    word_list = [i.strip().split('\t')[0] for i in lines]
    word_list = [f'@{i}$' for i in word_list]
    sents.append(''.join(word_list))

# ['HISTORY OF PRESENT ILLNESS : This patient is a 24 - year - old African - American female who presented to the hospital with buttock pain .']

In [31]:
def get_index(sent_file, note_file):
    with open(f'{sent_folder}/{sent_file}', 'r') as f:
        lines = f.readlines()
        sent = [i.strip().split('\t')[0] for i in lines]
    with open(f'{note_folder}/{note_file}', 'r') as f:
        lines = f.readlines()
        note = [i.strip().split('\t')[0] for i in lines]

    for index in range(len(note)):
        if note[index: index + len(sent)] == sent:
            return index, index + len(sent)

print(get_index('819.bio', 'sample_1074.bio'))

(0, 31)


In [32]:
output = pd.DataFrame()
error = pd.DataFrame()

for s in range(len(sent_files)):
    sent = sents[s]
    # if len(sent) < 7: # 如果句子过短不进行匹配，最短必须是7
    #     break
    flag = True
    for n in range(len(note_files)):
        note = notes[n]
        if sent in note:
            if flag == False:
                error.loc[len(error), 'sent_file_name'] = sent_files[s]
                error.loc[len(error)-1, 'note_file_name'] = note_files[n]
                error.loc[len(error)-1, 'sent'] = sents[s]
                error.loc[len(error)-1, 'note'] = notes[n]

            flag = False
            output.loc[len(output), 'sent_file_name'] = sent_files[s]
            output.loc[len(output)-1, 'note_file_name'] = note_files[n]
            index_tu = get_index(sent_files[s], note_files[n])
            # print(index_tu)
            # print(sent_files[s])
            # print(note_files[n])
            # print(sents[s])
            # print(notes[n])
            output.loc[len(output)-1, 'note_file_line_index_start'] = index_tu[0]
            output.loc[len(output)-1, 'note_file_line_index_end'] = index_tu[1]

    if flag:
        print(sent_files[s])
print('重复匹配到的句子', len(error))


重复匹配到的句子 350


In [33]:
joblib.dump(output, '/home/gy237/project/EHR_Foundation_model/note_2_sentence_level/output.joblib')
error.to_csv('/home/gy237/project/EHR_Foundation_model/note_2_sentence_level/error.csv')